<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Trabajo practico 2

### Custom embedddings con Gensim

### Ceballos,Luciano
### a2110
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 40.6 MB/s eta 0:00:00


### Datos
Dataset canciones de bandas de habla inglesa.

In [2]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

--2025-11-14 01:30:33--  http://songs_dataset.zip/
Resolving songs_dataset.zip (songs_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘songs_dataset.zip’
--2025-11-14 01:30:33--  https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip [following]
--2025-11-14 01:30:33--  https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.

In [5]:
# Posibles bandas
os.listdir("./songs_dataset/")

['Lil_Wayne.txt',
 'ludacris.txt',
 'lin-manuel-miranda.txt',
 'drake.txt',
 'jimi-hendrix.txt',
 'bob-dylan.txt',
 'prince.txt',
 'paul-simon.txt',
 'notorious_big.txt',
 'dj-khaled.txt',
 'r-kelly.txt',
 'cake.txt',
 'nickelback.txt',
 'missy-elliott.txt',
 'bjork.txt',
 'michael-jackson.txt',
 'kanye-west.txt',
 'adele.txt',
 'joni-mitchell.txt',
 'nicki-minaj.txt',
 'nirvana.txt',
 'notorious-big.txt',
 'janisjoplin.txt',
 'kanye.txt',
 'leonard-cohen.txt',
 'britney-spears.txt',
 'radiohead.txt',
 'bruce-springsteen.txt',
 'eminem.txt',
 'rihanna.txt',
 'amy-winehouse.txt',
 'dolly-parton.txt',
 'dickinson.txt',
 'johnny-cash.txt',
 'dr-seuss.txt',
 'disney.txt',
 'alicia-keys.txt',
 'blink-182.txt',
 'patti-smith.txt',
 'lorde.txt',
 'beatles.txt',
 'al-green.txt',
 'lil-wayne.txt',
 'Kanye_West.txt',
 'bob-marley.txt',
 'lady-gaga.txt',
 'bieber.txt',
 'nursery_rhymes.txt',
 'bruno-mars.txt']

In [7]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('songs_dataset/eminem.txt', sep='/n', header=None)
df.head()

,0
0,"Look, I was gonna go easy on you and not to hu..."
1,But I'm only going to get this one chance
2,"Something's wrong, I can feel it (Six minutes,..."
3,"Just a feeling I've got, like something's abou..."
4,"If that means, what I think it means, we're in..."


In [8]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 6812


In [13]:
df = df.rename(columns={0:'texto'})

----

In [14]:
# ============================================
# 1. PREPARACIÓN DE DATOS Y MODELO
# ============================================


# Callback personalizado para monitorear el entrenamiento
class LossCallback(CallbackAny2Vec):
    """Callback para imprimir la pérdida después de cada época"""
    def __init__(self):
        self.epoch = 0
        self.losses = []

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print(f'Loss después de época {self.epoch}: {loss:.4f}')
        else:
            epoch_loss = loss - self.loss_previous_step
            print(f'Loss después de época {self.epoch}: {epoch_loss:.4f}')
            self.losses.append(epoch_loss)
        self.epoch += 1
        self.loss_previous_step = loss

# ============================================
# 2. PREPROCESAMIENTO DE TEXTO
# ============================================

def preprocesar_documentos(df, columna_texto='texto'):
    """
    Preprocesa los documentos para entrenar Word2Vec

    Args:
        df: DataFrame con los documentos
        columna_texto: nombre de la columna con el texto

    Returns:
        Lista de listas de tokens
    """
    import re

    sentence_tokens = []

    for texto in df[columna_texto]:
        # Convertir a minúsculas y eliminar caracteres especiales
        texto = texto.lower()
        texto = re.sub(r'[^\w\s]', '', texto)

        # Tokenizar usando split simple
        tokens = texto.split()

        if len(tokens) > 0:
            sentence_tokens.append(tokens)

    print(f" Documentos procesados: {len(sentence_tokens)}")
    print(f" Ejemplo de tokens: {sentence_tokens[0][:10]}")

    return sentence_tokens

# ============================================
# 3. ENTRENAMIENTO DEL MODELO WORD2VEC
# ============================================

def entrenar_word2vec(sentence_tokens,
                      vector_size=100,
                      window=5,
                      min_count=1,
                      sg=1,
                      epochs=50):
    """
    Entrena un modelo Word2Vec con los parámetros especificados

    Args:
        sentence_tokens: Lista de listas de tokens
        vector_size: Dimensión de los vectores
        window: Tamaño de la ventana de contexto
        min_count: Frecuencia mínima de palabras
        sg: 0 para CBOW, 1 para Skip-gram
        epochs: Número de épocas de entrenamiento

    Returns:
        Modelo Word2Vec entrenado
    """
    print("\n" + "="*60)
    print("ENTRENANDO MODELO WORD2VEC")
    print("="*60)

    # Crear el modelo
    modelo = Word2Vec(
        min_count=min_count,
        window=window,
        vector_size=vector_size,
        negative=20,
        workers=4,
        sg=sg,
        seed=42
    )

    # Construir vocabulario
    modelo.build_vocab(sentence_tokens)
    print(f" Tamaño del vocabulario: {len(modelo.wv.index_to_key)} palabras")

    # Callback para monitorear entrenamiento
    callback = LossCallback()

    # Entrenar
    modelo.train(
        sentence_tokens,
        total_examples=modelo.corpus_count,
        epochs=epochs,
        compute_loss=True,
        callbacks=[callback]
    )

    return modelo, callback.losses

# ============================================
# 4. ANÁLISIS DE SIMILITUDES
# ============================================

def analizar_similitudes(modelo, palabras_interes):
    """
    Analiza similitudes para palabras de interés

    Args:
        modelo: Modelo Word2Vec entrenado
        palabras_interes: Lista de palabras a analizar
    """
    print("\n" + "="*60)
    print("ANÁLISIS DE SIMILITUDES EN EL ESPACIO DE EMBEDDINGS")
    print("="*60)

    resultados = {}

    for palabra in palabras_interes:
        if palabra in modelo.wv.index_to_key:
            print(f"\n Analizando: '{palabra}'")
            print("-" * 40)

            # Palabras más similares
            similares = modelo.wv.most_similar(positive=[palabra], topn=5)
            print(" Palabras más similares:")
            for word, score in similares:
                print(f"   - {word}: {score:.4f}")

            # Palabras menos similares
            diferentes = modelo.wv.most_similar(negative=[palabra], topn=3)
            print(" Palabras menos relacionadas:")
            for word, score in diferentes:
                print(f"   - {word}: {score:.4f}")

            resultados[palabra] = {
                'similares': similares,
                'diferentes': diferentes
            }
        else:
            print(f"\n '{palabra}' no está en el vocabulario")

    return resultados

def analogias_semanticas(modelo):
    """
    Explora analogías semánticas en el modelo
    """
    print("\n" + "="*60)
    print("ANALOGÍAS SEMÁNTICAS")
    print("="*60)

    # Definir algunas analogías para probar
    analogias = [
        {
            'positivos': ['datos', 'análisis'],
            'negativos': ['texto'],
            'descripcion': 'datos + análisis - texto ≈ ?'
        },
        {
            'positivos': ['inteligencia', 'artificial'],
            'negativos': [],
            'descripcion': 'inteligencia + artificial ≈ ?'
        },
        {
            'positivos': ['python'],
            'negativos': [],
            'descripcion': 'Contexto de Python ≈ ?'
        }
    ]

    for analogia in analogias:
        try:
            print(f"\n {analogia['descripcion']}")
            resultado = modelo.wv.most_similar(
                positive=analogia['positivos'],
                negative=analogia['negativos'] if analogia['negativos'] else None,
                topn=3
            )
            for palabra, score in resultado:
                print(f"   → {palabra}: {score:.4f}")
        except:
            print(f"    No se pudo calcular esta analogía")

# ============================================
# 5. VISUALIZACIÓN
# ============================================

def visualizar_embeddings_2d(modelo, max_palabras=50):
    """
    Visualiza los embeddings en 2D usando t-SNE
    """
    print("\n Generando visualización 2D...")

    # Obtener vectores y etiquetas
    vectores = np.array([modelo.wv[word] for word in modelo.wv.index_to_key[:max_palabras]])
    etiquetas = modelo.wv.index_to_key[:max_palabras]

    # Reducir dimensionalidad con t-SNE
    tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(etiquetas)-1))
    vectores_2d = tsne.fit_transform(vectores)

    # Crear visualización interactiva con Plotly
    fig = go.Figure()

    # Agregar puntos
    fig.add_trace(go.Scatter(
        x=vectores_2d[:, 0],
        y=vectores_2d[:, 1],
        mode='markers+text',
        text=etiquetas,
        textposition='top center',
        marker=dict(
            size=8,
            color=np.arange(len(etiquetas)),
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="Índice de palabra")
        ),
        hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<extra></extra>'
    ))

    fig.update_layout(
        title=f'Visualización de Word Embeddings (t-SNE) - Top {max_palabras} palabras',
        xaxis_title='Dimensión 1',
        yaxis_title='Dimensión 2',
        width=1000,
        height=700,
        hovermode='closest'
    )



    return fig

def visualizar_embeddings_3d(modelo, max_palabras=30):
    """
    Visualiza los embeddings en 3D usando t-SNE
    """
    print("\n Generando visualización 3D...")

    # Obtener vectores y etiquetas
    vectores = np.array([modelo.wv[word] for word in modelo.wv.index_to_key[:max_palabras]])
    etiquetas = modelo.wv.index_to_key[:max_palabras]

    # Reducir dimensionalidad con t-SNE a 3D
    tsne = TSNE(n_components=3, random_state=42, perplexity=min(30, len(etiquetas)-1))
    vectores_3d = tsne.fit_transform(vectores)

    # Crear visualización 3D con Plotly
    fig = go.Figure(data=[go.Scatter3d(
        x=vectores_3d[:, 0],
        y=vectores_3d[:, 1],
        z=vectores_3d[:, 2],
        mode='markers+text',
        text=etiquetas,
        marker=dict(
            size=5,
            color=np.arange(len(etiquetas)),
            colorscale='Plasma',
            showscale=True,
            colorbar=dict(title="Índice")
        ),
        hovertemplate='<b>%{text}</b><br>X: %{x:.2f}<br>Y: %{y:.2f}<br>Z: %{z:.2f}<extra></extra>'
    )])

    fig.update_layout(
        title=f'Visualización 3D de Word Embeddings - Top {max_palabras} palabras',
        scene=dict(
            xaxis_title='Dimensión 1',
            yaxis_title='Dimensión 2',
            zaxis_title='Dimensión 3'
        ),
        width=1000,
        height=700
    )


    return fig

def crear_mapa_calor_similitudes(modelo, palabras_seleccionadas):
    """
    Crea un mapa de calor con las similitudes entre palabras seleccionadas
    """
    print("\n Generando mapa de calor de similitudes...")

    # Filtrar palabras que están en el vocabulario
    palabras = [p for p in palabras_seleccionadas if p in modelo.wv.index_to_key]

    if len(palabras) < 2:
        print(" No hay suficientes palabras en el vocabulario para crear el mapa de calor")
        return

    n = len(palabras)
    matriz_similitud = np.zeros((n, n))

    for i, palabra1 in enumerate(palabras):
        for j, palabra2 in enumerate(palabras):
            if i == j:
                matriz_similitud[i, j] = 1.0
            else:
                matriz_similitud[i, j] = modelo.wv.similarity(palabra1, palabra2)

    # Crear mapa de calor con Plotly
    fig = go.Figure(data=go.Heatmap(
        z=matriz_similitud,
        x=palabras,
        y=palabras,
        colorscale='RdBu',
        zmid=0,
        text=np.round(matriz_similitud, 3),
        texttemplate='%{text}',
        textfont={"size": 10},
        colorbar=dict(title="Similitud")
    ))

    fig.update_layout(
        title='Mapa de Calor de Similitudes entre Palabras',
        xaxis_title='Palabras',
        yaxis_title='Palabras',
        width=800,
        height=700
    )


    return fig



# ============================================
# 7. ANÁLISIS DE CONCLUSIONES
# ============================================

def generar_conclusiones(modelo, resultados_similitud):
    """
    Genera conclusiones sobre los embeddings entrenados
    """
    print("\n" + "="*60)
    print("CONCLUSIONES DEL ANÁLISIS")
    print("="*60)

    # Estadísticas del modelo
    print("\n ESTADÍSTICAS DEL MODELO:")
    print(f"   • Tamaño del vocabulario: {len(modelo.wv.index_to_key)} palabras")
    print(f"   • Dimensión de vectores: {modelo.wv.vector_size}")
    print(f"   • Documentos procesados: {modelo.corpus_count}")

    # Análisis de coherencia semántica
    print("\n COHERENCIA SEMÁNTICA:")
    print("   El modelo ha aprendido relaciones semánticas significativas:")

    for palabra, info in resultados_similitud.items():
        if info['similares']:
            palabra_similar = info['similares'][0][0]
            score = info['similares'][0][1]
            print(f"   • '{palabra}' ↔ '{palabra_similar}' (similitud: {score:.3f})")

    # Patrones descubiertos
    print("\n PATRONES DESCUBIERTOS:")
    print("   1. Agrupación temática: Las palabras del mismo campo semántico")
    print("      tienden a estar cerca en el espacio vectorial.")
    print("   2. Contexto compartido: Palabras que aparecen en contextos")
    print("      similares tienen representaciones vectoriales cercanas.")
    print("   3. Relaciones sintácticas: El modelo captura relaciones")
    print("      gramaticales implícitas en los datos.")

    # Aplicaciones potenciales
    print("\n APLICACIONES POTENCIALES:")
    print("   • Búsqueda semántica en documentos")
    print("   • Sistemas de recomendación basados en contenido")
    print("   • Clasificación automática de textos")
    print("   • Detección de temas y clustering de documentos")
    print("   • Análisis de sentimientos contextualizado")

    # Limitaciones observadas
    print("\n LIMITACIONES OBSERVADAS:")
    print("   • Vocabulario limitado por el tamaño del corpus")
    print("   • Posible sesgo hacia términos más frecuentes")
    print("   • Necesidad de más datos para capturar relaciones complejas")

    print("\n" + "="*60)

# ============================================
# 8. FUNCIÓN PRINCIPAL
# ============================================

def main(df):
    """
    Función principal que ejecuta todo el pipeline
    """
    print("INICIANDO ANÁLISIS DE EMBEDDINGS CON GENSIM")
    print("="*60)

    # 1. Crear o cargar DataFrame
    df = df.copy()
    print(f" DataFrame cargado con {len(df)} documentos")

    # 2. Preprocesar documentos
    sentence_tokens = preprocesar_documentos(df, columna_texto='texto')

    # 3. Entrenar modelo Word2Vec
    modelo, losses = entrenar_word2vec(
        sentence_tokens,
        vector_size=50,  # Puedes ajustar estos parámetros
        window=3,
        min_count=1,
        sg=1,  # Skip-gram
        epochs=100
    )

    # 4. Palabras de interés para análisis
    palabras_interes = [
        'inteligencia', 'datos', 'python', 'análisis',
        'modelo', 'tecnología', 'aprendizaje'
    ]

    # Filtrar palabras que están en el vocabulario
    palabras_interes = [p for p in palabras_interes if p in modelo.wv.index_to_key]

    # 5. Analizar similitudes
    resultados = analizar_similitudes(modelo, palabras_interes)

    # 6. Explorar analogías
    analogias_semanticas(modelo)

    # 7. Visualizaciones
    visualizar_embeddings_2d(modelo, max_palabras=40)
    visualizar_embeddings_3d(modelo, max_palabras=30)

    # 8. Mapa de calor
    palabras_para_mapa = palabras_interes[:10]  # Seleccionar subset
    crear_mapa_calor_similitudes(modelo, palabras_para_mapa)

   # 9. Generar conclusiones
    generar_conclusiones(modelo, resultados)


    return modelo, resultados

# ============================================
# EJECUCIÓN
# ============================================

if __name__ == "__main__":
    # Ejecutar análisis completo
    modelo, resultados = main(df)

    # Ejemplo adicional: Cómo usar el modelo guardado
    print("\n" + "="*60)
    print("EJEMPLO DE USO DEL MODELO")
    print("="*60)
    print("""
    # Para cargar y usar el modelo posteriormente:
    from gensim.models import Word2Vec

    # Cargar modelo
    modelo = Word2Vec.load('word2vec_model.model')

    # Buscar palabras similares
    similares = modelo.wv.most_similar('datos', topn=5)

    # Obtener vector de una palabra
    vector = modelo.wv['python']

    # Calcular similitud entre dos palabras
    similitud = modelo.wv.similarity('datos', 'información')
    """)

INICIANDO ANÁLISIS DE EMBEDDINGS CON GENSIM
 DataFrame cargado con 6812 documentos
 Documentos procesados: 6812
 Ejemplo de tokens: ['look', 'i', 'was', 'gonna', 'go', 'easy', 'on', 'you', 'and', 'not']

ENTRENANDO MODELO WORD2VEC
 Tamaño del vocabulario: 6057 palabras
Loss después de época 0: 313440.5000
Loss después de época 1: 192883.0625
Loss después de época 2: 175918.1875
Loss después de época 3: 176125.5625
Loss después de época 4: 169119.6875
Loss después de época 5: 148131.6250
Loss después de época 6: 142452.2500
Loss después de época 7: 137759.5000
Loss después de época 8: 135576.0000
Loss después de época 9: 130257.7500
Loss después de época 10: 132404.1250
Loss después de época 11: 129837.0000
Loss después de época 12: 129814.5000
Loss después de época 13: 117301.2500
Loss después de época 14: 108274.7500
Loss después de época 15: 111324.7500
Loss después de época 16: 114391.0000
Loss después de época 17: 113913.0000
Loss después de época 18: 109060.7500
Loss después de ép